## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
ls

20240-0001-Codebook.pdf             20240-0001-Setup.dct
20240-0001-Data.dta                 20240-0001-Setup.do
20240-0001-Data.sav                 20240-0001-Setup.sas
20240-0001-Data.stc                 20240-0001-Setup.sps
20240-0001-Data.tsv                 20240-0001-Supplemental_syntax.do
20240-0001-Data.txt                 20240-0001-Supplemental_syntax.sas


In [12]:
pwd

'/Users/Winnifred/Desktop/Capstone/ICPSR_20240_RAWDATA/DS0001'

In [13]:
file_name = '/Users/Winnifred/Desktop/Capstone/ICPSR_20240_RAWDATA/DS0001/20240-0001-Data.tsv'

## Data

In [14]:
df = pd.read_csv(file_name, sep='\t', low_memory=False)

In [15]:
df.head()

,CASEID,V07312,V07319,V07320,V09154,V08825,V07477,V07292,V07297,V08249,...,V04611,NCSRWTSH,NCSRWTLG,NCNSWTSH,NCNSWTLG,NCNLWTSH,NCNLWTLG,NSNLWT,CPESWTSH,CPESWTLG
0,1,,,,1,2,0,0,2,2,...,,,,,,11627.4082,14040.66,15968.9517,11627.3949,14040.6279
1,2,,,,1,2,0,0,1,1,...,,,,,,10090.8728,12185.2188,13858.6912,10090.8613,12185.1909
2,3,,,,3,2,0,0,1,1,...,,,,,,10090.8728,12185.2188,13858.6912,10090.8613,12185.1909
3,4,,,,1,9,0,0,2,2,...,,,,,,5939.8128,6525.4281,7919.7406,5939.8017,6525.416
4,5,,,,1,6,0,0,1,1,...,,,,,,4572.4542,5023.2596,6096.5981,4572.4457,5023.2502


In [ ]:
df = df.set_index('CASEID')

In [ ]:
my_features = ['smoker']

In [ ]:
df['smoker'] = df.V00232

In [ ]:
df['smoker'].unique()

In [ ]:
# write a script that creates new feature columns for my dataframe

In [ ]:
df['smoker']